# Phase Picking CNN Algorithm

#### Paige Given and Albert Leonardo Aguilar Suarez - CS230

### Import Necessary Libraries

In [1]:
import numpy as np
import tensorflow as tf
import math
import h5py
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import glob
from obspy.core import read
import matplotlib.pyplot as plt
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

In [2]:
import pickle

In [378]:
#! pip install pandas
import pandas as pd

In [379]:
#! pip install sklearn

In [380]:
#!pip install seaborn

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

### Import and Divide Data

In [376]:
#import seismic data from filepath
seis_data=[]
for file in glob.glob('/Users/paigegiven/Desktop/CSProject/onebatch/*.pkl'):
    files = open(file,'rb')
    data=pickle.load(files)
    seis_data.append(data)
    #print(seis_data)

In [7]:
print(seis_data[1])

{'Pg': 0, 'Pn': 6000.0462, 'Sg': 0, 'Sn': 9161.0462, 'data': array([[ -1.39274057e-05,   1.37321310e-04,   8.75548201e-04, ...,
          8.71606798e-03,   9.65218914e-03,   1.01172338e-02],
       [  6.16421662e-05,   5.36179914e-04,   2.38116683e-03, ...,
         -3.94827006e-03,  -4.80134317e-03,  -6.19783777e-03],
       [  1.01817233e-04,   6.18889863e-04,   2.74197832e-03, ...,
          7.29245470e-03,   6.81605424e-03,   5.82166252e-03]])}


In [8]:
print(seis_data[1].items())

dict_items([('Pg', 0), ('Pn', 6000.0462), ('Sg', 0), ('Sn', 9161.0462), ('data', array([[ -1.39274057e-05,   1.37321310e-04,   8.75548201e-04, ...,
          8.71606798e-03,   9.65218914e-03,   1.01172338e-02],
       [  6.16421662e-05,   5.36179914e-04,   2.38116683e-03, ...,
         -3.94827006e-03,  -4.80134317e-03,  -6.19783777e-03],
       [  1.01817233e-04,   6.18889863e-04,   2.74197832e-03, ...,
          7.29245470e-03,   6.81605424e-03,   5.82166252e-03]]))])


In [9]:
#Create Pandas dataframe to more precisely analyze data
data = pd.DataFrame(list(seis_data),columns=['Pg','Pn','Sg','Sn','data'])
data=data.fillna('NaN')
print(data)

            Pg         Pn         Sg          Sn  \
0       0.0000  5999.5462     0.0000  10484.5462   
1       0.0000  6000.0462     0.0000   9161.0462   
2       0.0000  5998.0500     0.0000  11255.0500   
3       0.0000  5997.0500     0.0000  16337.0500   
4       0.0000  5999.1607     0.0000   7281.1607   
..         ...        ...        ...         ...   
159     0.0000  5999.1154     0.0000   9479.1154   
160     0.0000  5999.1609     0.0000   8424.1609   
161     0.0000  6000.5500     0.0000   9743.5500   
162     0.0000  6001.0462     0.0000   7213.0462   
163  5999.0462     0.0000  6524.0462      0.0000   

                                                  data  
0    [[-0.000205688020063, -0.000314050387823, -0.0...  
1    [[-1.39274057387e-05, 0.000137321309665, 0.000...  
2    [[-3.42587570399e-05, -0.00338175815329, -0.01...  
3    [[0.00291364712526, 0.0386191332504, 0.1483689...  
4    [[-6.29128131334e-05, -0.000173173379942, -0.0...  
..                               

In [31]:
#Check Input training data 
np.shape(data['data'][1])
print(len(data['data']))
data['data'][1]

164


array([[ -1.39274057e-05,   1.37321310e-04,   8.75548201e-04, ...,
          8.71606798e-03,   9.65218914e-03,   1.01172338e-02],
       [  6.16421662e-05,   5.36179914e-04,   2.38116683e-03, ...,
         -3.94827006e-03,  -4.80134317e-03,  -6.19783777e-03],
       [  1.01817233e-04,   6.18889863e-04,   2.74197832e-03, ...,
          7.29245470e-03,   6.81605424e-03,   5.82166252e-03]])

In [10]:
inputs = data['data']
print(inputs)

0      [[-0.000205688020063, -0.000314050387823, -0.0...
1      [[-1.39274057387e-05, 0.000137321309665, 0.000...
2      [[-3.42587570399e-05, -0.00338175815329, -0.01...
3      [[0.00291364712526, 0.0386191332504, 0.1483689...
4      [[-6.29128131334e-05, -0.000173173379942, -0.0...
                             ...                        
159    [[-0.00162438824888, -0.00938656555667, -0.044...
160    [[-0.00173031440833, -0.0124396915788, -0.0591...
161    [[0.016848594025, 0.147221356633, 0.5395635188...
162    [[4.01831746368e-06, 0.000117870272357, 0.0004...
163    [[6.18343233979e-05, -0.000796134270792, -0.00...
Name: data, Length: 164, dtype: object


In [156]:
#Iterate through to save each data matrix to a numpy array 
input_data=[]
for i in range(len(data['data'])):
    input_data.append(data['data'][i])
print(np.shape(input_data))

(164, 3, 30000)


In [12]:
#assign columns for labels 
labels = data[['Pg','Pn','Sg','Sn']]
print(labels)

            Pg         Pn         Sg          Sn
0       0.0000  5999.5462     0.0000  10484.5462
1       0.0000  6000.0462     0.0000   9161.0462
2       0.0000  5998.0500     0.0000  11255.0500
3       0.0000  5997.0500     0.0000  16337.0500
4       0.0000  5999.1607     0.0000   7281.1607
..         ...        ...        ...         ...
159     0.0000  5999.1154     0.0000   9479.1154
160     0.0000  5999.1609     0.0000   8424.1609
161     0.0000  6000.5500     0.0000   9743.5500
162     0.0000  6001.0462     0.0000   7213.0462
163  5999.0462     0.0000  6524.0462      0.0000

[164 rows x 4 columns]


In [64]:
#Divide data - 80% Training, 10% Evaluation, 10% Testing
X_train = input_data[:130]
print(np.shape(X_train))
X_dev = input_data[131:146]
print(np.shape(X_dev))
X_test = input_data[147:162]
print(np.shape(X_test))

(130, 3, 30000)
(15, 3, 30000)
(15, 3, 30000)


In [15]:
#Divide data - 80% Training, 10% Evaluation, 10% Testing
Y_train = labels[:130]
print(Y_train.shape)
Y_dev = labels[131:146]
print(Y_dev.shape)
Y_test = labels[147:162]
print(Y_test.shape)

(130, 4)
(15, 4)
(15, 4)


### Convert to tensorflow format

In [67]:
X_train_tf = tf.constant(X_train, dtype = tf.float32)
print(X_train_tf[1])
X_dev_tf = tf.constant(X_dev, dtype = tf.float32)
X_test_tf = tf.constant(X_test, dtype = tf.float32)

tf.Tensor(
[[ -1.39274061e-05   1.37321316e-04   8.75548227e-04 ...,   8.71606823e-03
    9.65218898e-03   1.01172337e-02]
 [  6.16421676e-05   5.36179927e-04   2.38116691e-03 ...,  -3.94826988e-03
   -4.80134320e-03  -6.19783765e-03]
 [  1.01817233e-04   6.18889870e-04   2.74197827e-03 ...,   7.29245460e-03
    6.81605423e-03   5.82166249e-03]], shape=(3, 30000), dtype=float32)


In [62]:
Y_train_tf = tf.constant(Y_train, dtype = tf.float32, shape=[130, 4])
print(Y_train_tf[1])
Y_dev_tf = tf.constant(Y_dev, dtype = tf.float32, shape=[15, 4])
Y_test_tf = tf.constant(Y_test, dtype = tf.float32, shape=[15, 4])

tf.Tensor([    0.      6000.0464     0.      9161.046 ], shape=(4,), dtype=float32)


### Set-up CNN Model

In [ ]:
#Using TensorFlow quickstart as guide: https://www.tensorflow.org/tutorials/quickstart/beginner

#ML Framework Inspired from: https://gitlab.com/fantine/ml-framework/-/blob/master/model/cnn2d_example.py

#ML Framework also Inspired from TensorFlow Lab in Coursera

In [360]:
#Create model framework
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.001, num_epochs = 100, minibatch_size = 32, print_cost = True):
    return tf.keras.models.Sequential([
        keras.layers.InputLayer(input_shape=((3,30000))),
        keras.layers.Conv1D(32,3, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(64,3, padding='same', activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(4)

    ])
    

In [361]:
#Assign model with tensorflow variables and hyperparameters
model = model(X_train_tf, Y_train_tf, X_test_tf, Y_test_tf, learning_rate = 0.001,num_epochs = 100, minibatch_size = 32, print_cost = True)

In [362]:
#visualize the model 
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 3, 32)             2880032   
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 32)             128       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 1, 32)             0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 1, 64)             6208      
_________________________________________________________________
flatten_32 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 260       
Total params: 2,886,628
Trainable params: 2,886,564
Non-trainable params: 64
__________________________________________

### Run Model on Training data to get Predictions

In [377]:
predictions = model(X_train_tf).numpy()
#print(predictions)

In [364]:
tf.nn.softmax(predictions).numpy()

array([[ 0.2501658 ,  0.24603955,  0.24506243,  0.2587322 ],
       [ 0.24388403,  0.24134567,  0.24672256,  0.26804778],
       [ 0.24970217,  0.25059757,  0.2362557 ,  0.26344448],
       [ 0.2517172 ,  0.2463471 ,  0.24576731,  0.25616837],
       [ 0.24830422,  0.2510098 ,  0.2511445 ,  0.2495415 ],
       [ 0.25101328,  0.2529467 ,  0.23051041,  0.2655296 ],
       [ 0.20623273,  0.2572878 ,  0.2533299 ,  0.28314957],
       [ 0.23725496,  0.2467975 ,  0.25198397,  0.26396355],
       [ 0.25375056,  0.26472855,  0.23645538,  0.24506548],
       [ 0.24130575,  0.24353091,  0.25419006,  0.2609733 ],
       [ 0.24536201,  0.2504181 ,  0.24227476,  0.26194516],
       [ 0.25435758,  0.24474901,  0.23695771,  0.26393566],
       [ 0.25126535,  0.2519293 ,  0.24853145,  0.24827392],
       [ 0.2555723 ,  0.24588275,  0.24543507,  0.25310996],
       [ 0.2158981 ,  0.26118076,  0.26883245,  0.25408873],
       [ 0.25159958,  0.24274547,  0.2724999 ,  0.2331551 ],
       [ 0.24738106,  0.

### Establish Loss Function

In [365]:
loss = tf.keras.losses.mape(Y_train_tf, predictions)
#loss = 100 * np.mean(abs((Y_train_tf - predictions) / Y_train_tf))

In [366]:
loss = tf.keras.losses.MeanAbsolutePercentageError(reduction=tf.keras.losses.Reduction.NONE)

In [367]:
loss(Y_train_tf, predictions).numpy()

array([ 23530682. ,   2892978.8,  13838616. ,   5980308.5,   2843493. ,
        21302510. ,  51421892. ,  17539548. ,  17648110. ,  13004409. ,
         3165619.2,  17714892. ,   3654002.2,  14897744. ,  54820490. ,
        10077434. ,   4896842.5,  29294476. ,  40279880. ,   6655779.5,
        22209856. ,  11911025. ,  33754936. ,  22597838. ,  26080112. ,
        32205496. ,  31691624. ,  49366264. ,  35947020. ,  52248172. ,
        22123134. ,  28433118. ,  22868224. ,  21645676. ,  21246476. ,
        38809004. ,  41016196. ,  36093536. ,  25257604. ,  17116178. ,
         1678726.4,   2404020.8,   7682583. ,   2799496.8,  14749694. ,
        30681104. ,   6400932.5,  37669436. ,  12079011. ,  39651250. ,
        12501758. ,  46615092. ,  31473122. ,  41754904. ,  44318908. ,
        43401964. ,   1950573. ,  59572588. ,   5374023.5,  20565074. ,
        29684000. ,   9398448. ,  13353295. ,  11219467. ,  56833212. ,
        52095212. ,   8023190. ,  11416148. ,  14762538. ,   631

### Configure Model for Training

In [368]:
model.compile(optimizer='adam',loss=loss,metrics=['accuracy'])

### Fit Model to Training Data

In [369]:
model.fit(X_train_tf, Y_train_tf, epochs=50)

Epoch 1/50
5/5 [==============================] - 1s 30ms/step - loss: 190916944.0000 - accuracy: 0.4462
Epoch 2/50
5/5 [==============================] - 0s 27ms/step - loss: 209816304.0000 - accuracy: 0.3308
Epoch 3/50
5/5 [==============================] - 0s 28ms/step - loss: 149085760.0000 - accuracy: 0.3846
Epoch 4/50
5/5 [==============================] - 0s 29ms/step - loss: 104438512.0000 - accuracy: 0.4769
Epoch 5/50
5/5 [==============================] - 0s 29ms/step - loss: 87257560.0000 - accuracy: 0.5077
Epoch 6/50
5/5 [==============================] - 0s 28ms/step - loss: 64592784.0000 - accuracy: 0.5231
Epoch 7/50
5/5 [==============================] - 0s 29ms/step - loss: 55610972.0000 - accuracy: 0.5231
Epoch 8/50
5/5 [==============================] - 0s 28ms/step - loss: 42896332.0000 - accuracy: 0.5308
Epoch 9/50
5/5 [==============================] - 0s 27ms/step - loss: 45224716.0000 - accuracy: 0.5000
Epoch 10/50
5/5 [==============================] - 0s 27ms/s

### Evaluate Model

#### Training Accuracy

In [370]:
model.evaluate(X_train_tf, Y_train_tf, verbose=1)

5/5 [==============================] - 0s 4ms/step - loss: 23315264.0000 - accuracy: 0.6000


[23315264.0, 0.6000000238418579]

#### Evaluation Accuracy

In [371]:
model.evaluate(X_dev_tf, Y_dev_tf, verbose=1)

1/1 [==============================] - 0s 19ms/step - loss: 17568976.0000 - accuracy: 0.6667


[17568976.0, 0.6666666865348816]